<a href="https://colab.research.google.com/github/brendan-mccaffrey/potfolio-optimization/blob/main/CommodityOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from cmath import nan
import numpy as np
import pandas as pd
!pip install mpt
from mpt import *
!pip3 install pickle5
import pickle5 as pickle

From raw commodities historical data, generate returns dataframe

In [ ]:
filepath = "./data/commodities.pkl"

with open(filepath, "rb") as dt:
  df = pickle.load(dt)

df.set_index("Date", inplace=True)

print("Original Dataframe")
print(df.head(5))

print("Returns Dataframe")
df = df.pct_change()
print(df.head(5))

Original Dataframe
           Gold (USD per Troy Ounce)  Silver (USD per Troy Ounce)  \
Date                                                                
1980-01-31                674.371338                    39.284318   
1980-02-29                665.289551                    35.598095   
1980-03-31                553.581055                    24.735476   
1980-04-30                517.409912                    14.978250   
1980-05-31                513.794922                    12.664350   
1980-06-30                600.716553                    15.807762   

            Platinum (USD per Troy Ounce)  Aluminum (USD/mt)  Copper (USD/mt)  \
Date                                                                            
1980-01-31                     806.786087        2054.860107      2592.633423   
1980-02-29                     882.315238        2131.008545      2916.711991   
1980-03-31                     746.544286        1978.378906      2303.827900   
1980-04-30             

Save returns dataframe to pickle file

In [ ]:
print("Saving to file")
filepath = "./data/commodity-returns.pkl"
with open(filepath, 'wb') as fp:
  pickle.dump(df, fp)

Saving to file


In [ ]:
# Check to see if commodity returns was saved successfully
filepath = "./data/commodity-returns.pkl"

with open(filepath, "rb") as dt:
  cdf = pickle.load(dt)
print(cdf.head(5))

            Gold (USD per Troy Ounce)  Silver (USD per Troy Ounce)  \
Date                                                                 
1980-01-31                        NaN                          NaN   
1980-02-29                  -0.013467                    -0.093834   
1980-03-31                  -0.167910                    -0.305146   
1980-04-30                  -0.065340                    -0.394463   
1980-05-31                  -0.006987                    -0.154484   

            Platinum (USD per Troy Ounce)  Aluminum (USD/mt)  Copper (USD/mt)  \
Date                                                                            
1980-01-31                            NaN                NaN              NaN   
1980-02-29                       0.093617           0.037058         0.125000   
1980-03-31                      -0.153880          -0.071623        -0.210128   
1980-04-30                      -0.210950          -0.023213        -0.099521   
1980-05-31             

Store asset names

In [ ]:
assets = cdf.columns

Perform weight optimization

In [ ]:
from numpy.linalg import inv

In [ ]:
# calculate historical mean return for each asset
E = np.array(cdf.mean(axis=0)).reshape(-1, 1)

# Calculate covariance matrix of assets' returns
cov_matrix = np.array(cdf.cov())

# Ones vector
ones = np.ones((E.shape[0], 1))

# Zeros
zeros = np.zeros((2, 2))

# Put together the A matrix
A = 2 * cov_matrix
A = np.append(A, E.T, axis=0)
A = np.append(A, ones.T, axis=0)
temp = np.append(E, ones, axis=1)
temp = np.append(temp, zeros, axis=0)
A = np.append(A, temp, axis=1)

# Put together the b vector
b = np.zeros((A.shape[1], 1))

# I set target return to be the expected return of commodities
b[A.shape[1] - 2][0] = E[0]
b[A.shape[1] - 1][0] = 1

# Now we want to optimize using matrix algebra
results = inv(A) @ b

# Grab first 4 elements of results because those are the weights
# Recall that we optimize across 4 assets so there are 4 weights
opt_W = results[: cdf.shape[1]]


optimal_weights = pd.DataFrame(opt_W, columns=["Optimal Weights"], index=assets)

## Check the result
This analytical approach allows for short positions.
If we only want positive weights we must use gradient descent

In [ ]:
# show the result
print(optimal_weights)

In [ ]:
for 

(503, 45)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline